# Train a U-Net for Cell Segmentation
To segment the full cell (cytosol + nucleus) we first train a U-Net to predict the foreground and boundaries of the cells. We will later uses these predictions to segment the cells using a seeded watershed from the nucleus predictions (see previous notebooks).

Here, we will use the pytorch library to train a U-Net using the images and ground-truth data we have downloaded. The goal of this lesson is learning how to use this library for training segmentation models.

Note: we are also working on a notebook that shows how to train a U-Net using only PyTorch in pytorch_train-cell-membrane-segmentation.ipynb. (This is not yet finished.)

In [ ]:
# General imports.

import itertools
import json
import os
import sys
import time
from glob import glob

import imageio.v3 as imageio
import matplotlib.pyplot as plt
from matplotlib import colors
import napari
import numpy as np
import skimage
from skimage.metrics import contingency_table, peak_signal_noise_ratio

import torch
from torchvision.transforms import v2

sys.path.append("..")
import utils

In [ ]:
# This function will download and unpack the data and do some further data preparation.
# It will only be executed if the data has not been downloaded yet.
data_dir = "../data"
if os.path.exists(data_dir):
    print("The data is downloaded already.")
else:
    utils.prepare_data(data_dir)

In [ ]:
import gc

model = None
train_loader = None
val_loader = None

with torch.no_grad():
    torch.cuda.empty_cache()

gc.collect()
torch.cuda.empty_cache()

### 1. Prepare the data for training

In [ ]:
# The data has been downloaded and separated into folders for the train, validation and test split already.
# We first create a dictionary with the location of the three different split folders.
data_dirs = {
    "train": os.path.join(data_dir, "train"),
    "val": os.path.join(data_dir, "val"),
    "test": os.path.join(data_dir, "test")
}

# And check the content for one of the samples.
# After the print you should see the images (marker, nuclei, serum), labels (cells and nuclei) as well as a json file.
train_sample0 = os.path.join(data_dirs["train"], "gt_image_000")
print(os.listdir(train_sample0))

In [ ]:
# Here we define the function that cuts out the small images centered around the cells for one image.

# This is a helper function for normalizing an image to the range [0, 1].
# Data normalization is important when training neural networks for image data
# to make sure all the inputs are in the same data range.
def normalize(image):
    image = image.astype("float32")
    #image = skimage.img_as_float(image)
    image -= image.min()
    image /= (image.max() + 1e-7)
    return image


# This is the main function for extracting the small images.
# We give it the path to the folder containing the data for a sample as input.
# Remember that this contains the images, segmentations and classification data
# in individual files for each sample. We have explored some of this data above.
def extract_images_and_labels(sample_folder):

    # We create the filepaths for the image and segmentation data we will load. 
    sample_name = os.path.basename(sample_folder)  # We can derive the sample name from the foldername.

    # serum is the image and cell is the label
    serum_path = os.path.join(sample_folder, f"{sample_name}_serum_image.tif")
    cell_segmentation_path = os.path.join(sample_folder, f"{sample_name}_cell_labels.tif")

    # And load the serum image and the cell segmentation.
    # We normalize the serum image so that the data range is in [0, 1].
    serum = normalize(imageio.imread(serum_path))
    # Note that we must not normalize the segmentation!
    cells = imageio.imread(cell_segmentation_path)
    # convert into binary mask
    # mask = (cells != 0)
    # mask[mask] = 1

    return serum, cells

In [ ]:
# This function applies the data extraction function we just defined to all samples
# for a split (training, validation or test) and extracts all the corresponding small images
# and classification labels.
def prepare_split(split):
    # Get all the folders for the samples of this split.
    split_folder = data_dirs[split]
    samples = glob(os.path.join(split_folder, "gt*"))
    # Iterate over all the samples and extract the images and labels from them.
    images, labels = [], []
    for sample in samples:
        sample_image, sample_labels = extract_images_and_labels(sample)
        images.append(sample_image)
        labels.append(sample_labels)
    return images, labels

In [ ]:
# Now we apply the functions for the training, validation and test split
# and check how many samples we have for each split.

train_images, train_labels = prepare_split("train")
print("We have", len(train_images), "training samples.")

In [ ]:
val_images, val_labels = prepare_split("val")
print("We have", len(val_images), "validation samples")

In [ ]:
test_images, test_labels = prepare_split("test")
print("We have", len(test_images), "test samples")

In [ ]:
viewer = napari.Viewer()
# # Add the image data: the marker channel (red channel) and nucleus channel (blue channel).
viewer.add_image(train_images[1], colormap="green", blending="additive")
viewer.add_labels(train_labels[1])

### 2. Build dataset, dataloader and model

In [ ]:
# Import the PyTorch functionality we need.
import torch
from torch.utils.data import Dataset, DataLoader

import tqdm
import torch.nn as nn
import torch.nn.functional as F

# Additional imports for evaluation and image transformations.
from sklearn import metrics
from skimage.transform import resize

In [ ]:
# We first check if we have access to a GPU.
# The model training will be much faster if we can use a GPU.
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is NOT available. The training will be very slow!")
    device = torch.device("cpu")

In [ ]:
# To provide the data for training we need to create a PyTorch Dataset.
# Datasets provide a single example (= small image + label) for training.
# They can also be used to process the data further. Here, we resize the images
# to the common shape within the dataset.

# You can find more information on datasets here:
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

# To create a dataset for our task we create a class that inherits
# from the PyTorch dataset.
class CustomDataset(Dataset):
    # Here we define the data for creating the dataset:
    # The small images for this dataset, the labels and the size for reshaping the images.
    def __init__(self, images, labels, crop_size, transform=None, mask_transform=None):
        self.images = images
        self.labels = labels
        self.crop_size = crop_size
        self.transform = transform
        self.mask_transform = mask_transform

    # A dataset needs a __len__ method that returns how many samples are in the dataset.
    # Here, the number of samples corresponds to the number of small images.
    def __len__(self):
        return len(self.images)

    # The __getitem__ method returns the image data and labels for a given sample index.
    def __getitem__(self, index):
        # Load the image data and label for this sample index.
        image = self.images[index]
        mask = self.labels[index]

        # crop the images, so that we can fit them into memory
        crop_shape = self.crop_size
        shape = image.shape
        if shape != crop_shape:
            assert image.ndim == mask.ndim == 2
            # bring it to common shape
            image = resize(image, shape, preserve_range=True)
            mask = resize(mask, shape, preserve_range=True)
            # Random crop same excerpt from image and mask
            i, j, h, w = v2.RandomCrop.get_params(
                torch.tensor(image), output_size=self.crop_size
            )
            image = v2.functional.crop(torch.tensor(image), i, j, h, w)
            mask = v2.functional.crop(torch.tensor(mask), i, j, h, w)

        # NOTE: transform would usually be applied to the image and mask so that we can
        # use data augmentations that act both on the image and the mask (e.g. rotations and flips)
        # apply transform if available
        if self.transform:
            image = self.transform(image)           
        if self.mask_transform:
            mask = self.mask_transform(mask)   
            
        # make sure to add the channel dimension to the image
        image, mask = np.array(image), np.array(mask)
        if image.ndim == 2:
            image = image[None]

        image = torch.tensor(image)
        mask = torch.tensor(mask)
        return image, mask


In [ ]:
# train the model for one epoch
def train_epoch(model, loader, loss, metric, optimizer):
    model.train()
    metric_list, loss_list = [], []
    for i, (x, y) in enumerate(loader):
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss_value = loss(pred, y)
        loss_value.backward()
        optimizer.step()
        loss_list.append(loss_value.item())
        if metric is not None:
            metric_value = metric(pred, y)
            metric_list.append(metric_value.item())

    if metric is not None:
        return np.mean(loss_list), np.mean(metric_list)
    else:
        return np.mean(loss_list), None


# validate the model
def validate(model, loader, loss, metric):
    model.eval()
    metric_list, loss_list = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss_value = loss(pred, y).item()
            loss_list.append(loss_value)
            if metric is not None:
                metric_value = metric(pred, y).item()
                metric_list.append(metric_value)

    if metric is not None:
        return np.mean(loss_list), np.mean(metric_list)
    else:
        return np.mean(loss_list), None


# run the whole training
def run_training(
    model, train_loader, val_loader, loss, metric, optimizer, n_epochs
):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []
    for epoch in tqdm.trange(n_epochs):
        epoch_train_loss, epoch_train_acc = train_epoch(model, train_loader, loss, metric, optimizer)
        epoch_val_loss, epoch_val_acc = validate(model, val_loader, loss, metric)
        
        # save the loss and accuracy for plotting
        train_losses.append(epoch_train_loss)
        train_accs.append(epoch_train_acc)
        val_losses.append(epoch_val_loss)
        val_accs.append(epoch_val_acc)
    
    return train_losses, train_accs, val_losses, val_accs


# plot the metrics
def plot(title, label, train_results, val_results, yscale='linear', save_path=None, 
         extra_pt=None, extra_pt_label=None):
    epoch_array = np.arange(len(train_results)) + 1
    train_label, val_label = "Training "+label.lower(), "Validation "+label.lower()
    
    #sns.set(style='ticks')

    plt.plot(epoch_array, train_results, epoch_array, val_results, linestyle='dashed', marker='o')
    legend = ['Train results', 'Validation results']
    
    if extra_pt:
        plt.plot(*extra_pt, 'ok')
    if extra_pt_label:
        legend.append(extra_pt_label)
        
    plt.legend(legend)
    plt.xlabel('Epoch')
    plt.ylabel(label)
    plt.yscale(yscale)
    plt.title(title)
    
    #sns.despine(trim=True, offset=5)
    plt.title(title, fontsize=15)
    if save_path:
        plt.savefig(str(save_path), bbox_inches='tight')
    plt.show()

In [ ]:
# a function to generate a random color map for a label image
def get_random_colors(labels):
    n_labels = len(np.unique(labels)) - 1
    cmap = [[0, 0, 0]] + np.random.rand(n_labels, 3).tolist()
    cmap = colors.ListedColormap(cmap)
    return cmap

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, last_activation=None):
        super().__init__()
        in_features = [in_channels, 64, 128, 256]
        out_features = [64, 128, 256, 512]
        self.encoders = nn.ModuleList([
            self.encoder_block(in_feats, out_feats, pool=level>0)
            for level, (in_feats, out_feats) in enumerate(zip(in_features, out_features))
        ])
        self.base = self.encoder_block(512, 1024, pool=True)
        self.last_activation = last_activation
        
        in_features = [1024, 512, 256, 128]
        out_features = [512, 256, 128, 64]
        self.decoders = nn.ModuleList([
            self.decoder_block(in_feats, out_feats)
            for in_feats, out_feats in zip(in_features, out_features)
        ])
        self.upsamplers = nn.ModuleList([
            nn.ConvTranspose2d(in_feats, out_feats, 2, stride=2)
            for in_feats, out_feats in zip(in_features, out_features)
        ])
        self.out_conv = nn.Conv2d(out_features[-1], out_channels, 1)
        
    def encoder_block(self, in_feats, out_feats, pool):
        layers = [nn.MaxPool2d(2)] if pool else []
        layers.extend([
            nn.Conv2d(in_feats, out_feats, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_feats, out_feats, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        ])
        return nn.Sequential(*layers)
    
    def decoder_block(self, in_feats, out_feats):
        return nn.Sequential(*[
            nn.Conv2d(in_feats, out_feats, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_feats, out_feats, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        ])
    
    def forward(self, x):
        from_encoder = []
        for encoder in self.encoders:
            x = encoder(x)
            from_encoder.append(x)
        x = self.base(x)
        from_encoder = from_encoder[::-1]
        for decoder, upsampler, from_enc in zip(self.decoders, self.upsamplers, from_encoder):
            x = decoder(torch.cat([
                from_enc, upsampler(x)
            ], dim=1))
        x = self.out_conv(x)
        if self.last_activation is not None:
            x = self.last_activation(x)
        return x

# Train with boundary channel
To avoid merges of touching cells, we will now add a boundary channel to the learning objective, and then use it for object separation in the instance segmentation funtion. To this end, we will train a U-Net that outputs 3 channels (background, foreground, object boundary) and use the cross entropy loss.

In [ ]:
# the issue should now be resolved. Let's try it out.
from scipy.ndimage import label
try:
    from skimage.segmentation import find_boundaries, watershed
except ImportError:
    fix_scipy_imports()
    from skimage.segmentation import find_boundaries, watershed

In [ ]:
# 0 for backgorund, 1 for foreground (= cells) and 2 for boundary (boundary pixel between cells and background or between 2 cells)
def label_transform(mask):
    fg_target = (mask > 0)
    bd_target = find_boundaries(mask)
    target = np.zeros(mask.shape, dtype="int64")
    # target[fg_target] = 1
    # target[bd_target] = 2
    return bd_target[None].astype("float64") # target

In [ ]:
crop_shape = (512, 512)

# Define your dataset with new transform
train_dataset = CustomDataset(train_images, train_labels, crop_size=crop_shape, mask_transform=label_transform)
val_dataset = CustomDataset(val_images, train_labels, crop_size=crop_shape, mask_transform=label_transform)

In [ ]:
viewer = napari.Viewer()

# You can ignore the commented code for now.
# image_vis, targets_vis = [], []
for ii, (image, target) in enumerate(train_dataset):
    viewer.add_image(image[0].numpy())
    viewer.add_labels(target[0].numpy().astype("uint8"))
    break
    
    # images_vis.append(image[0].numpy().squeeze())
    # targets_vis.append(target[0].numpy().squeeze())
    # if i >= 4: break

# for i, im in enumerate(images_vis):
#     viewer.add_image(im, name=f"image-{i}")

# for i, target in enumerate(targets_vis):
#     viewer.add_labels(target.astype("uint8"), name=f"target-{i}")

# viewer.grid.enabled = True

In [ ]:
viewer.grid.shape = (2, 2)

In [ ]:
print(viewer.shape()

In [ ]:
# visualize the new label transform and make sure it's correct
counter = 0
for im, target in train_dataset:
    if counter >= 3:
        break
    fig, ax = plt.subplots(1, 2, figsize=(10, 6))
    ax[0].axis("off")
    ax[0].imshow(im[0].squeeze(), cmap="gray")
    ax[1].axis("off")
    ax[1].imshow(target.squeeze())
    plt.show()
    counter += 1

In [ ]:
#instantiate the new unet and loss function
model = UNet(out_channels=1, last_activation=torch.nn.Sigmoid())
model.to(device)
# loss_function = nn.BCEWithLogitsLoss() # nn.CrossEntropyLoss()
# loss_function.to(device)

In [ ]:
from torch_em.model import UNet2d
model = UNet2d(in_channels=1, out_channels=1, final_activation="Sigmoid")
model = model.to(device)

In [ ]:
def flatten_samples(input_):
    # Get number of channels
    num_channels = input_.size(1)
    # Permute the channel axis to first
    permute_axes = list(range(input_.dim()))
    permute_axes[0], permute_axes[1] = permute_axes[1], permute_axes[0]
    # For input shape (say) NCHW, this should have the shape CNHW
    permuted = input_.permute(*permute_axes).contiguous()
    # Now flatten out all but the first axis and return
    flattened = permuted.view(num_channels, -1)
    return flattened


def dice_score(input_, target, eps=1e-7):
    assert input_.shape == target.shape, f"{input_.shape}, {target.shape}"
    # Flatten input and target to have the shape (C, N),
    # where N is the number of samples
    input_ = flatten_samples(torch.sigmoid(input_))
    target = flatten_samples(target)
    # Compute numerator and denominator (by summing over samples and
    # leaving the channels intact)
    numerator = (input_ * target).sum(-1)
    denominator = (input_ * input_).sum(-1) + (target * target).sum(-1)
    channelwise_score = 2 * (numerator / denominator.clamp(min=eps))
    # take the average score over the channels
    score = channelwise_score.mean()
    return score

In [ ]:
class DiceLoss(nn.Module):
    def forward(self, input_, target):
        return 1.0 - dice_score(input_, target)

In [ ]:
loss_function = DiceLoss()

In [ ]:
n_epochs = 15
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train_losses, _, val_losses, _ = run_training(model, train_loader, val_loader, loss_function, None, optimizer, n_epochs)
plot('Loss vs. Epoch', 'Loss', train_losses, val_losses)

In [ ]:
test_dataset = CustomDataset(test_images, test_labels, crop_size=crop_shape, mask_transform=label_transform)
model.eval()
for ii in range(4):
    im, mask = test_dataset[5]
    with torch.no_grad():
        pred = model(im[None].to(device)).cpu().numpy().squeeze()

        viewer = napari.Viewer()
        viewer.add_image(im.cpu().numpy().squeeze())
        viewer.add_image(pred)
        break
        
        # pred = torch.sigmoid(pred).cpu().numpy().squeeze()
        fig, ax = plt.subplots(1, 3, figsize=(12,7))
        ax[0].axis("off")
        ax[0].imshow(im[0], cmap="gray")
        ax[1].axis("off")
        ax[1].imshow(mask.squeeze(), interpolation="nearest")
        ax[2].axis("off")
        ax[2].imshow(pred, interpolation="nearest")
        # ax[3].axis("off")
        # ax[3].imshow(pred[1], interpolation="nearest")
        # ax[4].axis("off")
        # ax[4].imshow(pred[2], interpolation="nearest")
        plt.show()


### Implement and Use: Dice Loss Function
CrossEntropyLoss and BinaryCrossEntropyLoss do not return the desired results. We call the data "unbalanced" because the label we want to predict is disproportionately small in relation to the background (in particular the boundary channel). Therefore, it is beneficial to use a loss function that is robust against class imbalance:
In this part of the exercise we will use the Dice coefficient as loss.


Note that we do not need to predict a channel for background when using the dice coefficient as a loss (unlike with Cross Entropy).

In [ ]:
# disproportional
img, mask = test_dataset[0]
num_zeros = np.count_nonzero(mask == 0)
num_ones = np.count_nonzero(mask == 1)
print("Number of zeros:", num_zeros, "Number of ones:", num_ones, "ones/zeros:", num_ones/num_zeros)

In [ ]:
def label_transform(mask):
    mask = np.array(mask)
    fg_target = (mask > 0)[None].astype("float32")
    bd_target = find_boundaries(mask)[None].astype("float32")
    return np.concatenate([fg_target, bd_target])

In [ ]:
# Define your dataset with new transform
train_dataset = CustomDataset(train_images, train_labels, crop_size=crop_shape, mask_transform=label_transform)
val_dataset = CustomDataset(val_images, train_labels, crop_size=crop_shape, mask_transform=label_transform)

In [ ]:
# visualize the samples from the dataset and make sure the label transform is correct
counter = 0
for im, target in train_dataset:
    if counter > 3:
        break
    fig, ax = plt.subplots(1, 3, figsize=(16, 16))
    ax[0].axis("off")
    ax[0].imshow(im[0], cmap="gray")
    ax[1].axis("off")
    ax[1].imshow(target[0])
    ax[2].axis("off")
    ax[2].imshow(target[1])
    plt.show()
    counter += 1

In [ ]:
# override the forward function to add a sigmoid activation
# sigmoid activation is needed in combination with the new loss function: dice loss
class MyUNet(UNet):
    def forward(self, x):
        # call parent's forward method
        x = super().forward(x)
        x = torch.sigmoid(x)
        return x


In [ ]:
model = MyUNet(out_channels=2)
model_unet # UNet save trained
model.to(device)
loss = DiceLoss()
loss.to(device)

In [ ]:
# run the training again
n_epochs = 10
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True) 
val_loader = DataLoader(val_dataset, batch_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6) # default LR -3
metric = dice_score
train_losses, train_accs, val_losses, val_accs = run_training(model, train_loader, val_loader, loss, metric, optimizer, n_epochs)
plot('Loss vs. Epoch', 'Loss', train_losses, val_losses)
plot('Accuracy vs. Epoch', 'Accuracy', train_accs, val_accs)

In [ ]:
# check out instance segmentation for a few test images
counter = 0
with torch.no_grad():
    for im, mask in train_dataset:
        if counter > 3:
            break
        # predict with the model and apply sigmoid to map the prediction to the range [0, 1]
        pred = model(im[None].to(device))
        pred = torch.sigmoid(pred).cpu().numpy().squeeze()
        #pred = torch.tensor(pred).cpu().numpy().squeeze()
        #print( pred.shape)
        # get tbe nucleus instance segmentation by applying connected components to the binarized prediction
        fig, ax = plt.subplots(1, 5, figsize=(16, 16))
        ax[0].axis("off")
        ax[0].imshow(im.squeeze(), cmap="gray")
        ax[1].axis("off")
        ax[1].imshow(mask[0], cmap=get_random_colors(mask[0]), interpolation="nearest")
        ax[2].axis("off")
        ax[2].imshow(mask[1], cmap="gray")
        ax[3].axis("off")
        ax[3].imshow(pred[0], cmap=get_random_colors(pred[0]), interpolation="nearest")
        ax[4].axis("off")
        ax[4].imshow(pred[1], cmap=get_random_colors(pred[1]), interpolation="nearest")
        print( pred[0].mean())
        print( pred[1].mean())
        plt.show()
        counter += 1